In [28]:
%matplotlib inline
import shutil, os, sys, glob
import numpy as np
from matplotlib import pyplot as plt
import ipyvolume as ipv
#
sys.path.append('/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/src/')
import cryoemio
import imutils
import dataviz
import mrcutils

# some useful paths and parameters

In [29]:
input_dir = '/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/LocalRes/bin6_postautopick_good'
body_dir  = '/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin6_postautopick_good'
# files
keyword='bodies_bin6_postautopick_good_'
input_mrc    = input_dir+'/relion_locres_filtered.mrc'
output_mask  = body_dir+'/'+keyword
#

# load map and retrieve standard deviation

In [4]:
data = mrcutils.mrc2data(input_mrc)
input_std = np.std(data)
print(input_std)

0.11448731


# visualize effect of thresholding

In [52]:
data_thresh = mrcutils.mrc_select(input_mrc, mode='above_value', value=1*input_std)

In [53]:
ipv.quickvolshow(data_thresh, level=[1*input_std, 3*input_std], opacity=0.03, level_width=1*input_std, data_min=np.min(data_thresh), data_max=np.max(data_thresh))

# visualize effect of binarizing

In [54]:
mask = mrcutils.data2mask(data_thresh, sigma_blur=0, threshold=0.1)
ipv.quickvolshow(data_thresh, level=[0, 1], opacity=0.03)

# define bodies: segment differential thresholding map

The idea is to
- blur mask_dry a lot
- blur mask_fat a little
- define the main body as the intersection of both
- define the other bodies as what remains of mask_fat after removing the intersection
- then segment them

In other words:

$B_0 = M_{fat} \cap M_{dry}$

$\bigcup_{i>1}^{K} B_{i} = M_{fat} \setminus B_0 $

In [13]:
data_dry = mrcutils.mrc_select(input_mrc, mode='above_value', value=6*input_std)
mask_dry = mrcutils.data2mask(data_dry, sigma_blur=15, threshold=0.01)
data_fat = mrcutils.mrc_select(input_mrc, mode='above_value', value=1*input_std)
mask_fat = mrcutils.data2mask(data_fat, sigma_blur=1, threshold=0.1)

In [14]:
body0 = np.minimum(mask_dry,mask_fat)
bodyK = mask_fat - body0

In [15]:
ipv.quickvolshow(body0, level=[0, 1], opacity=0.03)

In [16]:
ipv.quickvolshow(bodyK, level=[0, 1], opacity=0.03)

In [17]:
mrcutils.data2mrc(output_mask+'body_0.mrc',body0,mrc_template=input_mrc)
mrcutils.data2mrc(output_mask+'body_K.mrc',bodyK,mrc_template=input_mrc)

In [21]:
import skimage.morphology as morphology
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from scipy import ndimage
fp=10
distance = ndimage.distance_transform_edt(bodyK)
local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((fp,fp,fp)), labels=bodyK)
markers = morphology.label(local_maxi)
labels_ws = watershed(-distance, markers, mask=bodyK)

In [22]:
ipv.quickvolshow(labels_ws, level=[0, float(np.max(labels_ws))], opacity=0.03)

In [24]:
mrcutils.data2mrc(output_mask+'body_K_watershed_seg.mrc',labels_ws,mrc_template=input_mrc)
# break down: one mask per segment
mrcutils.seg2mask(output_mask+'body_K_watershed_seg.mrc', output_mask+'body_K_watershed', sigma_blur=1,sort='volume',verbose=True)

/opt/rh/rh-python36/root/usr/lib64/python3.6/site-packages/numpy/core/numeric.py:538: UserWarning: Warning: converting a masked element to nan.
  return array(a, dtype, copy=False, order=order)


/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin6_postautopick_good/bodies_bin6_postautopick_good_body_K_watershed1.mrc > volume = 60090.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin6_postautopick_good/bodies_bin6_postautopick_good_body_K_watershed2.mrc > volume = 54080.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin6_postautopick_good/bodies_bin6_postautopick_good_body_K_watershed3.mrc > volume = 50832.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin6_postautopick_good/bodies_bin6_postautopick_good_body_K_watershed4.mrc > volume = 39704.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin6_postautopick_good/bodies_bin6_postautopick_good_body_K_watershed5.mrc > volume = 29656.0
/gpfs/slac/cryo/fs1/u/fpoitevi/processing/20181005-rib-TEM4/bodymaker/bin6_postautopick_good/bodies_bin6_postautopick_good_body_K_watershed6.mrc > volume = 21258.0
/gpfs/slac/cryo/

At that point, a visual inspection (in Chimera) leads to following decision:
- body 0: 0,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
- body 1: 1
- body 2: 2
- body 3: 3
- body 4: 4
- body 5: 5
- body 6: 6

In [30]:
# final body 0 
shutil.copyfile(output_mask+'body_0.mrc', 'tmp.mrc')
for i in [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]:
    mrcutils.mrc_algebra('tmp.mrc',output_mask+'body_K_watershed'+str(i)+'.mrc','tmp2.mrc')
    os.remove('tmp.mrc')
    os.rename('tmp2.mrc', 'tmp.mrc')
mrcutils.mrc2mask('tmp.mrc', output_mask+'body_0_final.mrc', sigma_blur=0., threshold=0.1)
os.remove('tmp.mrc')
# final bodies 1 to 6
for i in np.arange(1,7):
    shutil.copyfile(output_mask+'body_K_watershed'+str(i)+'.mrc', output_mask+'body_'+str(i)+'_final.mrc')
# final body 1
#shutil.copyfile(output_mask+'body_K_watershed1.mrc', 'tmp.mrc')
#for i in [7]:
#    mrcutils.mrc_algebra('tmp.mrc',output_mask+'body_K_watershed'+str(i)+'.mrc','tmp2.mrc')
#    os.remove('tmp.mrc')
#    os.rename('tmp2.mrc', 'tmp.mrc')
#mrcutils.mrc2mask('tmp.mrc', output_mask+'body_1_final.mrc', sigma_blur=0., threshold=0.1)
#os.remove('tmp.mrc')
# final body 2
#shutil.copyfile(output_mask+'body_K_watershed2.mrc', output_mask+'body_2_final.mrc')
# final body 3
#shutil.copyfile(output_mask+'body_K_watershed3.mrc', 'tmp.mrc')
#for i in [4,5,6,9]:
#    mrcutils.mrc_algebra('tmp.mrc',output_mask+'body_K_watershed'+str(i)+'.mrc','tmp2.mrc')
#    os.remove('tmp.mrc')
#    os.rename('tmp2.mrc', 'tmp.mrc')
#mrcutils.mrc2mask('tmp.mrc', output_mask+'body_3_final.mrc', sigma_blur=0., threshold=0.1)
#os.remove('tmp.mrc')
# final body 4
#shutil.copyfile(output_mask+'body_K_watershed11.mrc', output_mask+'body_4_final.mrc')

OSError: [Errno 122] Disk quota exceeded